In [1]:
import dask
import dask.threaded
import dask.multiprocessing
from dask.distributed import Client

c = Client()


/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:74: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)


In [2]:
c

Client Scheduler: tcp://127.0.0.1:34028 Dashboard: http://127.0.0.1:37333/status,Cluster Workers: 8 Cores: 48 Memory: 134.91 GB


In [3]:
import xarray as xr 
import dask 
import numpy as np 
import os 
import time 
import glob
from datetime import date
import numpy.ma as ma

today=date.today()

#pip install windrose 
from windrose import WindroseAxes
import matplotlib.pyplot as plt


import seaborn as sns
sns.set(color_codes=True)

#https://pmall.univ-grenoble-alpes.fr/docs/auto_examples_code/windrose/windrose-exemple.html

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/windrose/windrose.py:29: MatplotlibDeprecationWarning: 
The Appender class was deprecated in Matplotlib 3.1 and will be removed in 3.3.
  addendum = docstring.Appender(msg, "\n\n")
/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/windrose/windrose.py:30: MatplotlibDeprecationWarning: 
The copy_dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use docstring.copy() and cbook.dedent() instead.
  return lambda func: addendum(docstring.copy_dedent(base)(func))
/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/windrose/windrose.py:30: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.getdoc() instead.
  return lambda func: addendum(docstring.copy_dedent(base)(func))
/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/windrose/windrose.py:30: MatplotlibDeprec

In [5]:
def dx_var(data,e1):
    dx_var = (data.shift(x=-1) - data)/e1
    return dx_var
def dy_var(data,e2):
    dy_var = (data.shift(y=-1) - data)/e2
    return dy_var
    

In [6]:
data_dir = '/store/molines/NATL60/NATL60-CJM165-S/1d/'
gridfile='/store/molines/NATL60/NATL60-I/NATL60_v4.1_cdf_mesh_hgr.nc'
dsgrid=xr.open_dataset(gridfile,chunks={'x':1000,'y':1000})

In [7]:
e1=dsgrid['e1t'][0]
e2=dsgrid['e2t'][0]


In [ ]:
imin_box = {'AC' : 3088, 'NE' :3838, 'GS' : 994}
imax_box = {'AC' : 3386, 'NE' :4119, 'GS' : 1295}
jmin_box = {'AC' : 301, 'NE' :1783, 'GS' : 751}
jmax_box = {'AC' : 657, 'NE' :2239, 'GS' : 1155}


In [15]:
for month in ['march', 'september']:
#for month in ['september']:
    
    if month == 'march':
        Date=['20130301','20130302','20130303','20130304','20130305','20130306','20130307','20130308','20130309','20130310','20130311','20130312','20130313','20130314','20130315','20130316','20130317','20130318','20130319','20130320','20130321','20130322','20130323','20130324','20130325','20130326','20130327','20130328','20130329','20130330']
    if month == 'september':
        Date=['20120901','20120902','20120903','20120904','20120905','20120906','20120907','20120908','20120909','20120910','20120911','20120912','20120913','20120914','20120915','20120916','20120917','20120918','20120919','20120920','20120921','20120922','20120923','20120924','20120925','20120926','20120927','20120928','20120929','20120930']

    for date in Date:
        print(date)
        
        year=date[0:4]
        month=date[4:6]
        day=date[6:]

        filename = sorted(glob.glob(data_dir+str(year)+'/NATL60-CJM165_y'+str(year)+'m'+str(month)+'d'+str(day)+'.1d_gridTsurf.nc'))
        file=filename[0]
        print(file)

        ds=xr.open_dataset(file,chunks={'x':1000,'y':1000,'time_counter':1})
        dataT=ds['sosstsst']
        dataS=ds['sosaline']
        dataT_dx=dx_var(dataT,e1)
        dataS_dx=dx_var(dataS,e1)
        dataT_dy=dy_var(dataT,e1)
        dataS_dy=dy_var(dataS,e1)
        R=(dataT_dx + 1j * dataT_dy)/(dataS_dx + 1j * dataS_dy)
        magR=np.abs(R)
        magR_ma=ma.masked_invalid(magR)
        Tu=np.arctan(magR)
        
#        for box in ['AC','NE','GS']:
        for box in [NE','GS']:
            
            print('compute for box '+str(box))

            magR_box=magR[:,jmin[box]:jmax[box],imin[box]:imax[box]].stack(z=('x', 'y','time_counter'))
            Tu_box=Tu[:,jmin[box]:jmax[box],imin[box]:imax[box]].stack(z=('x', 'y','time_counter'))
        
            magR_box_nonan=magR_box[~np.isnan(magR_box)]
            Tu_box_nonan=Tu_box[~np.isnan(magR_box)]
            magR_box_nonan_noinf=magR_box_nonan[~np.isinf(magR_box_nonan)]
            Tu_box_nonan_noinf=Tu_box_nonan[~np.isinf(magR_box_nonan)]
            magR_box_nonan_noinf=magR_box_nonan_noinf.reset_index('z')
            Tu_box_nonan_noinf=Tu_box_nonan_noinf.reset_index('z')

            print('write /scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1d/turner-angle/NATL60-CJM165_magR_'+str(date)+'.1d_'+str(box)+'.nc')
            dataset_mag=magR_box_nonan_noinf.to_dataset(name='magR')
            dataset_mag.attrs['global_attribute']= 'Stack of absolute value of ratio between temperature and salinity gradient of data for day '+str(date)
            dataset_mag.to_netcdf(path='/scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1d/turner-angle/NATL60-CJM165_magR_'+str(date)+'.1d_'+str(box)+'.nc',mode='w')
    
            print('write /scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1d/turner-angle/NATL60-CJM165_Tu_'+str(date)+'.1d_'+str(box)+'.nc')
            dataset_mag=Tu_box_nonan_noinf.to_dataset(name='Tu')
            dataset_mag.attrs['global_attribute']= 'Stack of turner angle of data for day '+str(date)
            dataset_mag.to_netcdf(path='/scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1d/turner-angle/NATL60-CJM165_Tu_'+str(date)+'.1d_'+str(box)+'.nc',mode='w')


20120901
/store/molines/NATL60/NATL60-CJM165-S/1d/2012/NATL60-CJM165_y2012m09d01.1d_gridTsurf.nc
write /scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1d/turner-angle/NATL60-CJM165_magR_20120901.1d_AC.nc
write /scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1d/turner-angle/NATL60-CJM165_Tu_201209011d__AC.nc
20120902
/store/molines/NATL60/NATL60-CJM165-S/1d/2012/NATL60-CJM165_y2012m09d02.1d_gridTsurf.nc
write /scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1d/turner-angle/NATL60-CJM165_magR_20120902.1d_AC.nc
write /scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1d/turner-angle/NATL60-CJM165_Tu_201209021d__AC.nc
20120903
/store/molines/NATL60/NATL60-CJM165-S/1d/2012/NATL60-CJM165_y2012m09d03.1d_gridTsurf.nc


distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)


write /scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1d/turner-angle/NATL60-CJM165_magR_20120903.1d_AC.nc
write /scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1d/turner-angle/NATL60-CJM165_Tu_201209031d__AC.nc
20120904
/store/molines/NATL60/NATL60-CJM165-S/1d/2012/NATL60-CJM165_y2012m09d04.1d_gridTsurf.nc


distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)


write /scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1d/turner-angle/NATL60-CJM165_magR_20120904.1d_AC.nc
write /scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1d/turner-angle/NATL60-CJM165_Tu_201209041d__AC.nc
20120905
/store/molines/NATL60/NATL60-CJM165-S/1d/2012/NATL60-CJM165_y2012m09d05.1d_gridTsurf.nc


...

distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)


write /scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1d/turner-angle/NATL60-CJM165_magR_20120929.1d_AC.nc
write /scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1d/turner-angle/NATL60-CJM165_Tu_201209291d__AC.nc
20120930
/store/molines/NATL60/NATL60-CJM165-S/1d/2012/NATL60-CJM165_y2012m09d30.1d_gridTsurf.nc


distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)


write /scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1d/turner-angle/NATL60-CJM165_magR_20120930.1d_AC.nc
write /scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1d/turner-angle/NATL60-CJM165_Tu_201209301d__AC.nc


distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.nanny - WARNING - Worker process 34962 was killed by signal 9
distributed.nanny - WARNING - Worker process 34971 was killed by signal 9
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
